# 🏠 Analyse Exploratoire et Prédiction des Prix Immobiliers

## Projet d'Analyse de Données - Math IA

**Auteur:** Projet Math IA  
**Date:** Juin 2025  
**Objectif:** Développer un modèle de régression multiple pour prédire les prix des biens immobiliers

---

## 📋 Objectifs du Projet

1. **Collecte des données** : Scraper les annonces immobilières
2. **Nettoyage des données** : Construire une pipeline de traitement
3. **Analyse exploratoire** : Explorer les relations entre variables
4. **Modélisation** : Développer un modèle de régression multiple
5. **Évaluation** : Analyser les performances et l'importance des variables

---

## 📊 Structure du Notebook

1. Import des bibliothèques requises
2. Chargement et inspection des données
3. Prétraitement des données
4. Ingénierie des features
5. Sélection et entraînement des modèles
6. Évaluation des modèles
7. Optimisation des hyperparamètres
8. Sauvegarde et chargement du modèle

## 1. Import des Bibliothèques Requises

Importation de toutes les bibliothèques nécessaires pour l'analyse de données, la visualisation et la modélisation.

In [ ]:
# Manipulation de données
import pandas as pd
import numpy as np
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Statistiques
from scipy import stats
from scipy.stats import pearsonr

# Utilitaires
import joblib
from typing import Dict, List, Tuple
from tqdm import tqdm

# Configuration des graphiques
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

# Ajout du répertoire parent au path pour importer nos modules
sys.path.append('..')

print("✅ Toutes les bibliothèques ont été importées avec succès!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn version: {sklearn.__version__}")

## 2. Chargement et Inspection des Données

Dans cette section, nous chargeons les données immobilières et effectuons une inspection initiale pour comprendre la structure et la qualité des données.

In [ ]:
# Chargement des données
data_path = '../data/raw_properties.csv'

# Vérification de l'existence du fichier
if not os.path.exists(data_path):
    print("❌ Fichier de données non trouvé!")
    print("Exécutez d'abord le script de collecte de données ou le script principal.")
    print("Alternative: génération de données d'exemple...")
    
    # Génération de données d'exemple si le fichier n'existe pas
    from src.data_scraper import generate_sample_data
    df_raw = generate_sample_data(200)
    os.makedirs('../data', exist_ok=True)
    df_raw.to_csv(data_path, index=False)
    print("✅ Données d'exemple générées et sauvegardées!")
else:
    # Chargement des données existantes
    df_raw = pd.read_csv(data_path)
    print("✅ Données chargées avec succès!")

print(f"📊 Nombre de propriétés: {len(df_raw)}")
print(f"📈 Nombre de variables: {len(df_raw.columns)}")
print(f"📅 Période de données: {pd.Timestamp.now().strftime('%Y-%m-%d')}")

# Affichage des premières lignes
print("\n🔍 Aperçu des données:")
df_raw.head()

In [ ]:
# Inspection détaillée des données
print("📋 INFORMATIONS SUR LE DATASET")
print("=" * 50)

# Informations générales
print(f"Forme du dataset: {df_raw.shape}")
print(f"Taille mémoire: {df_raw.memory_usage(deep=True).sum() / 1024:.1f} KB")

print("\n📊 TYPES DE DONNÉES:")
print(df_raw.dtypes)

print("\n🔍 INFORMATIONS DÉTAILLÉES:")
df_raw.info()

print("\n📈 STATISTIQUES DESCRIPTIVES:")
df_raw.describe()

In [ ]:
# Analyse des valeurs manquantes
print("🚨 ANALYSE DES VALEURS MANQUANTES")
print("=" * 50)

missing_values = df_raw.isnull().sum()
missing_percent = (missing_values / len(df_raw)) * 100

missing_df = pd.DataFrame({
    'Colonnes': missing_values.index,
    'Valeurs_manquantes': missing_values.values,
    'Pourcentage': missing_percent.values
}).sort_values('Pourcentage', ascending=False)

print(missing_df)

# Visualisation des valeurs manquantes
if missing_values.sum() > 0:
    plt.figure(figsize=(12, 6))
    
    # Graphique en barres des valeurs manquantes
    plt.subplot(1, 2, 1)
    missing_cols = missing_df[missing_df['Valeurs_manquantes'] > 0]
    plt.bar(missing_cols['Colonnes'], missing_cols['Pourcentage'], color='red', alpha=0.7)
    plt.title('Pourcentage de Valeurs Manquantes par Colonne')
    plt.ylabel('Pourcentage (%)')
    plt.xticks(rotation=45)
    
    # Heatmap des valeurs manquantes
    plt.subplot(1, 2, 2)
    sns.heatmap(df_raw.isnull(), cbar=True, yticklabels=False, cmap='viridis')
    plt.title('Heatmap des Valeurs Manquantes')
    
    plt.tight_layout()
    plt.show()
else:
    print("✅ Aucune valeur manquante détectée!")

## 3. Prétraitement des Données

Le prétraitement est une étape cruciale qui inclut:
- Nettoyage des données (suppression des doublons, gestion des outliers)
- Gestion des valeurs manquantes
- Validation des données selon des règles métier
- Normalisation et standardisation

In [ ]:
# Utilisation de notre pipeline de nettoyage personnalisé
from src.data_pipeline import DataCleaner

print("🧹 NETTOYAGE DES DONNÉES")
print("=" * 50)

# Initialisation du nettoyeur
cleaner = DataCleaner()

# Nettoyage des données
df_cleaned = cleaner.clean_data(df_raw)

# Rapport de nettoyage
cleaning_report = cleaner.generate_cleaning_report(df_raw, df_cleaned)

print("📊 RAPPORT DE NETTOYAGE:")
print(f"• Lignes originales: {cleaning_report['original_rows']}")
print(f"• Lignes nettoyées: {cleaning_report['cleaned_rows']}")
print(f"• Lignes supprimées: {cleaning_report['rows_removed']} ({cleaning_report['removal_percentage']:.1f}%)")
print(f"• Nouvelles features créées: {len(cleaning_report['new_features'])}")

if cleaning_report['new_features']:
    print("• Features ajoutées:")
    for feature in cleaning_report['new_features']:
        print(f"  - {feature}")

print("\n✅ Données nettoyées avec succès!")
print(f"📊 Nouvelles dimensions: {df_cleaned.shape}")

# Aperçu des données nettoyées
print("\n🔍 Aperçu des données nettoyées:")
df_cleaned.head()

## 4. Analyse Exploratoire des Données (EDA)

L'analyse exploratoire nous permet de:
- Comprendre la distribution des variables
- Identifier les corrélations entre variables
- Détecter les patterns et tendances du marché immobilier
- Préparer les insights pour la modélisation

In [ ]:
# 4.1 Distribution des prix
print("💰 ANALYSE DE LA DISTRIBUTION DES PRIX")
print("=" * 50)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Distribution des Prix Immobiliers', fontsize=16, fontweight='bold')

# Histogramme des prix
axes[0, 0].hist(df_cleaned['prix_dh'], bins=50, color='skyblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Histogramme des Prix')
axes[0, 0].set_xlabel('Prix (DH)')
axes[0, 0].set_ylabel('Fréquence')
axes[0, 0].ticklabel_format(style='scientific', axis='x', scilimits=(0,0))

# Box plot des prix
bp = axes[0, 1].boxplot(df_cleaned['prix_dh'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightcoral')
axes[0, 1].set_title('Box Plot des Prix')
axes[0, 1].set_ylabel('Prix (DH)')
axes[0, 1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# Distribution log des prix
prix_log = np.log10(df_cleaned['prix_dh'])
axes[1, 0].hist(prix_log, bins=50, color='lightgreen', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Distribution des Prix (échelle log)')
axes[1, 0].set_xlabel('Log10(Prix)')
axes[1, 0].set_ylabel('Fréquence')

# Q-Q plot
stats.probplot(df_cleaned['prix_dh'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Distribution normale)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistiques de prix
print(f"📊 Statistiques des prix:")
print(f"• Prix moyen: {df_cleaned['prix_dh'].mean():,.0f} DH")
print(f"• Prix médian: {df_cleaned['prix_dh'].median():,.0f} DH")
print(f"• Prix minimum: {df_cleaned['prix_dh'].min():,.0f} DH")
print(f"• Prix maximum: {df_cleaned['prix_dh'].max():,.0f} DH")
print(f"• Écart-type: {df_cleaned['prix_dh'].std():,.0f} DH")
print(f"• Coefficient de variation: {(df_cleaned['prix_dh'].std() / df_cleaned['prix_dh'].mean()):.2%}")

In [ ]:
# 4.2 Analyse des corrélations
print("\n🔗 ANALYSE DES CORRÉLATIONS")
print("=" * 50)

# Sélection des variables numériques
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
correlation_matrix = df_cleaned[numeric_cols].corr()

plt.figure(figsize=(12, 10))

# Création de la heatmap
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
           mask=mask,
           annot=True, 
           cmap='RdBu_r', 
           center=0,
           square=True,
           fmt='.2f',
           cbar_kws={"shrink": .8})

plt.title('Matrice de Corrélation des Variables Numériques', 
         fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Corrélations avec le prix
price_correlations = correlation_matrix['prix_dh'].abs().sort_values(ascending=False)
print("📊 Corrélations avec le prix (valeurs absolues):")
for var, corr in price_correlations.items():
    if var != 'prix_dh':
        print(f"• {var}: {corr:.3f}")

# Variables les plus corrélées avec le prix
top_correlations = price_correlations.drop('prix_dh').head(5)
print(f"\n🏆 Top 5 des variables les plus corrélées avec le prix:")
for i, (var, corr) in enumerate(top_correlations.items(), 1):
    print(f"{i}. {var}: {corr:.3f}")

In [ ]:
# 4.3 Analyse par localisation et type de bien
print("\n🗺️ ANALYSE PAR LOCALISATION ET TYPE DE BIEN")
print("=" * 50)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Analyse par Localisation et Type de Bien', fontsize=16, fontweight='bold')

# Prix moyen par ville
prix_par_ville = df_cleaned.groupby('localisation')['prix_dh'].agg(['mean', 'count']).sort_values('mean', ascending=False)

bars1 = axes[0, 0].bar(range(len(prix_par_ville)), prix_par_ville['mean'], 
                      color='steelblue', alpha=0.7)
axes[0, 0].set_title('Prix Moyen par Ville')
axes[0, 0].set_xlabel('Ville')
axes[0, 0].set_ylabel('Prix Moyen (DH)')
axes[0, 0].set_xticks(range(len(prix_par_ville)))
axes[0, 0].set_xticklabels(prix_par_ville.index, rotation=45)
axes[0, 0].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# Nombre de propriétés par ville
bars2 = axes[0, 1].bar(range(len(prix_par_ville)), prix_par_ville['count'], 
                      color='orange', alpha=0.7)
axes[0, 1].set_title('Nombre de Propriétés par Ville')
axes[0, 1].set_xlabel('Ville')
axes[0, 1].set_ylabel('Nombre de Propriétés')
axes[0, 1].set_xticks(range(len(prix_par_ville)))
axes[0, 1].set_xticklabels(prix_par_ville.index, rotation=45)

# Distribution par type de bien
if 'type_bien' in df_cleaned.columns:
    type_counts = df_cleaned['type_bien'].value_counts()
    wedges, texts, autotexts = axes[1, 0].pie(type_counts.values, labels=type_counts.index, 
                                             autopct='%1.1f%%', startangle=90)
    axes[1, 0].set_title('Répartition par Type de Bien')
    
    # Prix moyen par type de bien
    prix_par_type = df_cleaned.groupby('type_bien')['prix_dh'].mean().sort_values(ascending=True)
    bars3 = axes[1, 1].barh(range(len(prix_par_type)), prix_par_type.values,
                           color='lightcoral', alpha=0.7)
    axes[1, 1].set_title('Prix Moyen par Type de Bien')
    axes[1, 1].set_xlabel('Prix Moyen (DH)')
    axes[1, 1].set_yticks(range(len(prix_par_type)))
    axes[1, 1].set_yticklabels(prix_par_type.index)
    axes[1, 1].ticklabel_format(style='scientific', axis='x', scilimits=(0,0))

plt.tight_layout()
plt.show()

# Insights par localisation
print("🏙️ Insights par localisation:")
ville_plus_chere = prix_par_ville.index[0]
prix_plus_cher = prix_par_ville['mean'].iloc[0]
ville_moins_chere = prix_par_ville.index[-1]
prix_moins_cher = prix_par_ville['mean'].iloc[-1]

print(f"• Ville la plus chère: {ville_plus_chere} ({prix_plus_cher:,.0f} DH)")
print(f"• Ville la moins chère: {ville_moins_chere} ({prix_moins_cher:,.0f} DH)")
print(f"• Écart de prix: {((prix_plus_cher - prix_moins_cher) / prix_moins_cher * 100):.1f}%")

if 'type_bien' in df_cleaned.columns:
    print(f"\n🏠 Insights par type de bien:")
    type_plus_cher = prix_par_type.index[-1]
    type_moins_cher = prix_par_type.index[0]
    print(f"• Type le plus cher: {type_plus_cher} ({prix_par_type.iloc[-1]:,.0f} DH)")
    print(f"• Type le moins cher: {type_moins_cher} ({prix_par_type.iloc[0]:,.0f} DH)")

## 5. Sélection et Entraînement des Modèles

Dans cette section, nous:
- Préparons les données pour la modélisation
- Testons plusieurs algorithmes de régression
- Comparons leurs performances
- Sélectionnons le meilleur modèle

In [ ]:
# 5.1 Préparation des données pour la modélisation
print("🤖 PRÉPARATION DES DONNÉES POUR LA MODÉLISATION")
print("=" * 60)

# Utilisation de notre prédicteur personnalisé
from src.modeling import RealEstatePricePredictor

# Initialisation du prédicteur
predictor = RealEstatePricePredictor()

# Préparation des données
X_train, X_test, y_train, y_test = predictor.prepare_data(df_cleaned)

print(f"✅ Données préparées pour {len(predictor.feature_names)} features")
print(f"📊 Taille d'entraînement: {X_train.shape}")
print(f"📊 Taille de test: {X_test.shape}")

# 5.2 Entraînement des modèles
print("\n🚀 ENTRAÎNEMENT DES MODÈLES")
print("=" * 60)

# Entraînement de tous les modèles
results = predictor.train_models(X_train, y_train, X_test, y_test)

# Sélection du meilleur modèle
best_model_name = predictor.select_best_model()

print(f"\n🏆 Meilleur modèle sélectionné: {best_model_name}")

# Récapitulatif des performances
print("\n📊 RÉCAPITULATIF DES PERFORMANCES:")
print("-" * 40)
performance_df = pd.DataFrame()

for name, result in results.items():
    performance_df = pd.concat([performance_df, pd.DataFrame({
        'Modèle': [name],
        'R²': [result['test_metrics']['r2']],
        'RMSE': [result['test_metrics']['rmse']],
        'MAE': [result['test_metrics']['mae']],
        'MAPE (%)': [result['test_metrics']['mape']]
    })], ignore_index=True)

# Tri par R² décroissant
performance_df = performance_df.sort_values('R²', ascending=False)
print(performance_df.round(4))

## 6. Évaluation des Modèles

Nous évaluons les modèles selon plusieurs critères:
- **R²** : Coefficient de détermination (plus proche de 1 = meilleur)
- **RMSE** : Root Mean Square Error (plus bas = meilleur)
- **MAE** : Mean Absolute Error (plus bas = meilleur)
- **MAPE** : Mean Absolute Percentage Error (plus bas = meilleur)

In [ ]:
# 6.1 Visualisation des performances des modèles
print("📊 VISUALISATION DES PERFORMANCES")
print("=" * 50)

# Graphique de comparaison des modèles
predictor.plot_model_comparison(save=False)

# 6.2 Analyse détaillée du meilleur modèle
print(f"\n🔍 ANALYSE DÉTAILLÉE - {best_model_name}")
print("=" * 50)

# Graphique prédictions vs réelles
predictor.plot_predictions_vs_actual(X_test, y_test, save=False)

# Analyse de l'importance des features
print("\n📈 IMPORTANCE DES FEATURES")
print("=" * 40)

importance_df = predictor.analyze_feature_importance()
if not importance_df.empty:
    print("Top 10 des features les plus importantes:")
    top_features = importance_df.head(10)
    for i, (_, row) in enumerate(top_features.iterrows(), 1):
        print(f"{i:2d}. {row['feature']:<25} : {row['importance']:.4f}")
    
    # Graphique d'importance
    predictor.plot_feature_importance(save=False)
else:
    print("❌ Importance des features non disponible pour ce modèle")

# 6.3 Métriques détaillées du meilleur modèle
best_metrics = results[best_model_name]['test_metrics']
train_metrics = results[best_model_name]['train_metrics']

print(f"\n📊 MÉTRIQUES DÉTAILLÉES - {best_model_name}")
print("=" * 50)
print("Performances sur les données de TEST:")
print(f"• R² Score        : {best_metrics['r2']:.4f}")
print(f"• RMSE           : {best_metrics['rmse']:,.0f} DH")
print(f"• MAE            : {best_metrics['mae']:,.0f} DH")
print(f"• MAPE           : {best_metrics['mape']:.2f}%")

print("\nPerformances sur les données d'ENTRAÎNEMENT:")
print(f"• R² Score        : {train_metrics['r2']:.4f}")
print(f"• RMSE           : {train_metrics['rmse']:,.0f} DH")
print(f"• MAE            : {train_metrics['mae']:,.0f} DH")
print(f"• MAPE           : {train_metrics['mape']:.2f}%")

# Détection d'overfitting
r2_diff = train_metrics['r2'] - best_metrics['r2']
if r2_diff > 0.1:
    print(f"\n⚠️  ATTENTION: Possible overfitting détecté!")
    print(f"   Différence R² (train - test): {r2_diff:.3f}")
else:
    print(f"\n✅ Pas d'overfitting détecté (différence R²: {r2_diff:.3f})")

## 7. Optimisation des Hyperparamètres

L'optimisation des hyperparamètres permet d'améliorer les performances du modèle en trouvant les meilleurs paramètres pour chaque algorithme.

In [ ]:
# 7.1 Optimisation des hyperparamètres
print("⚙️  OPTIMISATION DES HYPERPARAMÈTRES")
print("=" * 60)

# Optimisation pour les modèles sélectionnés
optimized_models = predictor.hyperparameter_tuning(X_train, y_train)

if optimized_models:
    print("\n📊 RÉSULTATS DE L'OPTIMISATION:")
    print("-" * 40)
    
    for name, result in optimized_models.items():
        print(f"\n🔧 {name}:")
        print(f"   Meilleurs paramètres: {result['best_params']}")
        print(f"   Score CV RMSE: {np.sqrt(result['best_score']):,.0f} DH")
        
        # Comparaison avec le modèle de base
        base_rmse = results[name]['test_metrics']['rmse']
        optimized_rmse = np.sqrt(result['best_score'])
        improvement = ((base_rmse - optimized_rmse) / base_rmse) * 100
        
        if improvement > 0:
            print(f"   📈 Amélioration: {improvement:.1f}%")
        else:
            print(f"   📉 Dégradation: {abs(improvement):.1f}%")
else:
    print("⏭️  Optimisation désactivée pour cette démo (peut prendre du temps)")

# 7.2 Test du modèle optimisé (simulation)
print(f"\n🎯 PERFORMANCE DU MODÈLE FINAL")
print("=" * 50)

# Utilisation du meilleur modèle actuel
final_model = predictor.best_model
final_predictions = final_model.predict(X_test)

# Calcul des métriques finales
final_r2 = r2_score(y_test, final_predictions)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
final_mae = mean_absolute_error(y_test, final_predictions)
final_mape = np.mean(np.abs((y_test - final_predictions) / y_test)) * 100

print(f"Modèle final: {best_model_name}")
print(f"• R² Score : {final_r2:.4f}")
print(f"• RMSE     : {final_rmse:,.0f} DH")
print(f"• MAE      : {final_mae:,.0f} DH")
print(f"• MAPE     : {final_mape:.2f}%")

# Interprétation des résultats
print(f"\n💡 INTERPRÉTATION:")
if final_r2 > 0.8:
    print("• 🟢 Excellent: Le modèle explique plus de 80% de la variance des prix")
elif final_r2 > 0.6:
    print("• 🟡 Bon: Le modèle explique plus de 60% de la variance des prix")
elif final_r2 > 0.4:
    print("• 🟠 Moyen: Le modèle explique plus de 40% de la variance des prix")
else:
    print("• 🔴 Faible: Le modèle explique moins de 40% de la variance des prix")

print(f"• En moyenne, le modèle se trompe de {final_mae:,.0f} DH sur le prix")
print(f"• L'erreur relative moyenne est de {final_mape:.1f}%")

## 8. Sauvegarde et Chargement du Modèle

Cette section montre comment persister le modèle entraîné et le recharger pour une utilisation future.

In [ ]:
# 8.1 Sauvegarde du modèle
print("💾 SAUVEGARDE DU MODÈLE")
print("=" * 40)

# Création du répertoire models
os.makedirs('../models', exist_ok=True)

# Sauvegarde du meilleur modèle
model_path = '../models/best_price_predictor.pkl'
predictor.save_model(model_path)

print(f"✅ Modèle sauvegardé: {model_path}")
print(f"📊 Modèle: {best_model_name}")
print(f"🎯 Performance (R²): {final_r2:.4f}")

# 8.2 Test du chargement du modèle
print(f"\n📥 TEST DU CHARGEMENT DU MODÈLE")
print("=" * 40)

# Création d'un nouveau prédicteur pour tester le chargement
new_predictor = RealEstatePricePredictor()
new_predictor.load_model(model_path)

print(f"✅ Modèle rechargé: {new_predictor.best_model_name}")

# Test de prédiction avec le modèle rechargé
test_prediction = new_predictor.best_model.predict(X_test[:1])
print(f"🧪 Test de prédiction: {test_prediction[0]:,.0f} DH")

# 8.3 Exemple de prédiction pratique
print(f"\n🏠 EXEMPLE DE PRÉDICTION PRATIQUE")
print("=" * 40)

# Propriété d'exemple
example_property = {
    'surface_m2': 120,
    'nombre_chambres': 3,
    'localisation': 'Casablanca',
    'type_bien': 'Appartement',
    'annee_construction': 2018
}

print("Caractéristiques de la propriété:")
for key, value in example_property.items():
    print(f"• {key}: {value}")

# Note: La prédiction nécessiterait un preprocessing complet
print(f"\n💰 Prix estimé: [Nécessite preprocessing complet]")
print("   (Utilisez la fonction predict_price() du module modeling)")

print(f"\n🎉 ANALYSE TERMINÉE AVEC SUCCÈS!")
print("=" * 60)
print("📊 Résumé du projet:")
print(f"• Données analysées: {len(df_cleaned)} propriétés")
print(f"• Meilleur modèle: {best_model_name}")
print(f"• Performance finale (R²): {final_r2:.4f}")
print(f"• Erreur moyenne: {final_mae:,.0f} DH")
print(f"• Modèle sauvegardé: {model_path}")

print(f"\n🚀 Prochaines étapes recommandées:")
print("• Collecter plus de données réelles")
print("• Ajouter des features géographiques (distance centre-ville, etc.)")
print("• Tester des modèles plus avancés (XGBoost, Neural Networks)")
print("• Déployer le modèle en production")
print("• Créer une interface utilisateur pour les prédictions")